In [1]:
import unittest
from itertools import product
import copy
from shapely import LineString, Point
import geopandas as gpd
import geo_nx as gnx 
import networkx as nx 
import pandas as pd
import geo_nx.utils as utils
from networkx.utils import graphs_equal

paris = Point(2.3514, 48.8575)
lyon = Point(4.8357, 45.7640)
marseille = Point(5.3691, 43.3026)
bordeaux = Point(-0.56667, 44.833328)

SLICE = 10000

In [32]:

simplemap = gpd.GeoDataFrame({'geometry': [LineString([paris, lyon]), LineString([lyon, marseille]), 
    LineString([paris, bordeaux]), LineString([bordeaux, marseille])]}, crs=4326).to_crs(2154)
gr_simple = gnx.from_geopandas_edgelist(simplemap)
gr_simple.nodes[0]['city'] = 'paris'
gr_simple.nodes[1]['ville'] = 'lyon'
dgr_simple = gr_simple.to_directed()
dgr_cop = dgr_simple.copy()
nx.remove_edge_attributes(dgr_cop, "geometry")
dgr_cop.edges[0, 1]

{'weight': 391542.8889183051}

In [29]:
dgr_simple.edges[0, 1]

{'geometry': <LINESTRING (652411.148 6862135.813, 842666.659 6519924.367)>,
 'weight': 391542.8889183051}

In [4]:
def macro_node(dgr, node, uturn=False):
    e_nodes = list(range(node * SLICE + 2, node * SLICE + 2 * (1 + len(list(dgr.neighbors(node)))), 2))
    s_nodes = list(range(node * SLICE + 1, node * SLICE + 1 + 2 * len(list(dgr.neighbors(node))), 2))
    i_edges = list(product(e_nodes, s_nodes))
    if not uturn:
        i_edges = [edge for edge in i_edges if edge[0] != edge[1] + 1]
    return (e_nodes, s_nodes, i_edges)

In [5]:
def to_macro_node(dgr, node, uturn=False):
    e_nodes, s_nodes, i_edges = macro_node(dgr, node, uturn)
    pred_nodes = list(dgr.predecessors(node))
    succ_nodes = list(dgr.successors(node))
    for e_node, pnd in zip(e_nodes, pred_nodes):
        dgr.remove_edge(pnd, node)
        dgr.add_edges_from([(pnd, e_node, {'truc':1, 'mach': 'in'})])
        #dgr.add_edge(pnd, e_node, {'truc':1, 'mach': 'in'})
    for s_node, snd in zip(s_nodes, succ_nodes):
        dgr.remove_edge(node, snd)
        dgr.add_edges_from([(s_node, snd, {'truc':1, 'mach': 'in'})])
        #dgr.add_edge(s_node, snd, {'truc':1, 'mach': 'in'})    
    dgr.add_edges_from(i_edges)
    dgr.remove_node(node)
    dg_node = dgr.subgraph(e_nodes + s_nodes)
    nx.set_node_attributes(dg_node, 'connecteur', 'nature')
    nx.set_node_attributes(dg_node, None, 'geometry')
    nx.set_edge_attributes(dg_node, 'connexion', 'nature')
    nx.set_edge_attributes(dg_node, None, 'geometry')
    nx.set_edge_attributes(dg_node, 0, 'weight')
    nx.set_edge_attributes(dg_node, None, 'NATIONALRO')
    return (e_nodes, s_nodes, i_edges)

In [6]:
def to_undirected_edges(edges):
    ext_edges = [(edge[0] // SLICE, edge[1] // SLICE) for edge in edges if edge[0] // SLICE != edge[1] // SLICE]
    return sorted(list(set([(min(edge), max(edge)) for edge in ext_edges])))
    

In [7]:
2103 // 100; 2103 % 100; min((3,2))
sorted(list(set( [ (5,6), (1,3), (5,6), (6,5), (1,2), (5,3)])))

[(1, 2), (1, 3), (5, 3), (5, 6), (6, 5)]

In [8]:
gr = nx.Graph()
gr.add_edges_from([(1, 2), (3, 1), (3,4), (2,3)])
dgr = gr.to_directed()
dgr.edges

OutEdgeView([(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 4), (3, 2), (4, 3)])

In [9]:
gr.edges

EdgeView([(1, 2), (1, 3), (2, 3), (3, 4)])

In [10]:
gr.edges[1,2]

{}

In [11]:
nodes = list(dgr.nodes)
for node in nodes:
    to_macro_node(dgr, node)
nodes

[1, 2, 3, 4]

In [12]:
dgr.nodes

NodeView((10002, 10004, 10001, 10003, 20002, 20004, 20001, 20003, 30002, 30004, 30006, 30001, 30003, 30005, 40002, 40001))

In [13]:
dgr.edges

OutEdgeView([(10002, 10003), (10004, 10001), (10001, 20004), (10003, 30004), (20002, 20003), (20004, 20001), (20001, 30006), (20003, 10002), (30002, 30003), (30002, 30005), (30004, 30001), (30004, 30005), (30006, 30001), (30006, 30003), (30001, 40002), (30003, 10004), (30005, 20002), (40001, 30002)])

In [14]:
nx.shortest_path(gr, source=2, target=4, weight='weight')

[2, 3, 4]

In [15]:
nx.shortest_path(dgr, source=20001, target=40002) #, weight='weight')

[20001, 30006, 30001, 40002]

In [16]:
for edge in dgr.edges:
    print(dgr.edges[edge])

{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'truc': 1, 'mach': 'in'}
{'truc': 1, 'mach': 'in'}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'truc': 1, 'mach': 'in'}
{'truc': 1, 'mach': 'in'}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None}
{'truc': 1, 'mach': 'in'}
{'truc': 1, 'mach': 'in'}
{'truc': 1, 'mach': 'in'}
{'truc': 1, 'mach': 'in'}


In [17]:
(dgr.edges[10002, 10003], dgr.edges[10003, 30004] )

({'nature': 'connexion', 'geometry': None, 'weight': 0, 'NATIONALRO': None},
 {'truc': 1, 'mach': 'in'})

In [18]:
print(to_undirected_edges(dgr.edges), list(gr.edges))
to_undirected_edges(dgr.edges) == list(gr.edges)

[(1, 2), (1, 3), (2, 3), (3, 4)] [(1, 2), (1, 3), (2, 3), (3, 4)]


True

In [19]:

def di_neighbors(dgr, node, uturn=False, nopath=None):
    e_nodes = [str(node) + 'e'+ str(pnd) for pnd in dgr.predecessors(node)]
    s_nodes = [str(node) + 's'+ str(snd) for snd in dgr.successors(node)]
    m_edges = list(product(e_nodes, s_nodes))
    if not uturn:
        m_edges = [edge for edge in m_edges if edge[0].replace("e", "s") != edge[1]]
    if nopath:
        no_edges = [ (str(node) + 'e' + str(edge[0]), str(node) + 's' + str(edge[1])) for edge in nopath]
        return list(set(m_edges) - set(no_edges))
    return m_edges

In [20]:
def insert_di_nodes(dgr, node, uturn=False, nopath=None):
    m_edges = di_neighbors(dgr, node, uturn, nopath)
    pred_nodes = list(dgr.predecessors(node))
    succ_nodes = list(dgr.successors(node))
    for pnd in pred_nodes:
        dgr.remove_edge(pnd, node)
        dgr.add_edge(pnd, str(node) + 'e' + str(pnd))
    for snd in succ_nodes:
        dgr.remove_edge(node, snd)
        dgr.add_edge(str(node) + 's' + str(snd), snd)    
    dgr.add_edges_from(m_edges)

In [21]:
"123e456".replace("e", "s")

'123s456'

In [22]:
gr = nx.Graph()
gr.add_edges_from([(1, 2), (1, 3), (3,4), (2,3)])
nx.shortest_path(gr, source=2, target=4)

[2, 3, 4]

In [23]:
dgr = gr.to_directed()
dgr.edges

OutEdgeView([(1, 2), (1, 3), (2, 1), (2, 3), (3, 1), (3, 4), (3, 2), (4, 3)])

In [24]:
nx.shortest_path(dgr, source=2, target=4)

[2, 3, 4]

In [25]:
di_neighbors(dgr, 3)

[('3e1', '3s4'),
 ('3e1', '3s2'),
 ('3e2', '3s1'),
 ('3e2', '3s4'),
 ('3e4', '3s1'),
 ('3e4', '3s2')]

In [26]:
di_neighbors(dgr, 3, nopath=[(2,4)])

[('3e2', '3s1'),
 ('3e1', '3s4'),
 ('3e1', '3s2'),
 ('3e4', '3s2'),
 ('3e4', '3s1')]

In [27]:
dgr = gr.to_directed()
insert_di_nodes(dgr, 3)
nx.shortest_path(dgr, source=2, target=4), nx.shortest_path(dgr, source=4, target=2)

([2, '3e2', '3s4', 4], [4, '3e4', '3s2', 2])

In [28]:
dgr = gr.to_directed()
insert_di_nodes(dgr, 3, nopath=[(2,4)])
nx.shortest_path(dgr, source=2, target=4), nx.shortest_path(dgr, source=4, target=2)

([2, 1, '3e1', '3s4', 4], [4, '3e4', '3s2', 2])